# CNN

### 1. Обучение классификатора картинок на примере CIFAR-100 (датасет можно изменить) сверточной сетью (самописной)

In [20]:
# Сделаем необходимые импорты
import torch
from torch import nn
import torchvision
import torch.nn.functional as F
from torch import optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from tqdm import tqdm

# игнорируем предупреждения
import warnings
warnings.filterwarnings("ignore")

In [2]:
use_cuda = torch.cuda.is_available()

device = torch.device("cuda:0" if use_cuda else "cpu")
print(device)

cuda:0


In [3]:
class MyOwnCifar(torch.utils.data.Dataset):
   
    def __init__(self, init_dataset, transform=None):
        self._base_dataset = init_dataset
        self.transform = transform

    def __len__(self):
        return len(self._base_dataset)

    def __getitem__(self, idx):
        img = self._base_dataset[idx][0]
        if self.transform is not None:
            img = self.transform(img)
        return img, self._base_dataset[idx][1]

In [4]:
# Загрузим датасет CIFAR-100, сразу же создадим dataloader для него
# Если вам не хватает вычислительных ресурсов, то можно вернуться к CIFAR-10
train_dataset = torchvision.datasets.CIFAR100(root='data/',
                                             train=True,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True)

print(train_dataset)
print('='*85)
print(test_dataset)

train_dataset = MyOwnCifar(train_dataset, transforms.ToTensor())
test_dataset = MyOwnCifar(test_dataset, transforms.ToTensor())

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting data/cifar-100-python.tar.gz to data/
Files already downloaded and verified
Dataset CIFAR100
    Number of datapoints: 50000
    Root location: data/
    Split: Train
Dataset CIFAR100
    Number of datapoints: 10000
    Root location: ./data
    Split: Test


In [5]:
train_loader =  torch.utils.data.DataLoader(train_dataset,
                                            batch_size=64, 
                                            shuffle=True)

test_loader =  torch.utils.data.DataLoader(test_dataset, 
                                           batch_size=16,
                                           shuffle=False)

In [6]:
# напишем сверточную сеть
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__() 
        self.dp_three = nn.Dropout(0.2)
        self.dp_four = nn.Dropout(0.2)
        
        self.bn_one = torch.nn.BatchNorm2d(3) 
        self.conv_one = torch.nn.Conv2d(3, 30, 3)
        self.bn_two = torch.nn.BatchNorm2d(30) 
        self.conv_two = torch.nn.Conv2d(30, 60, 3)
        self.bn_three = torch.nn.BatchNorm2d(60)
        self.conv_three = torch.nn.Conv2d(60, 120, 3)
        self.bn_four = torch.nn.BatchNorm2d(120)
        self.fc1 = torch.nn.Linear(480, 20)
        self.fc2 = torch.nn.Linear(20, 60)
        self.out = torch.nn.Linear(60, 100)
        
        
    def forward(self, x):
        x = self.bn_one(x)
        x = self.conv_one(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_two(x)
        x = self.conv_two(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_three(x)
        x = self.conv_three(x)
        x = F.leaky_relu(x, 0.1)
        x = F.max_pool2d(x, 2)
        
        x = self.bn_four(x)
        x = x.view(x.size(0), -1)
        x = self.dp_three(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp_four(x)
        x = self.fc2(x)
        x = F.relu(x)
        return self.out(x)
       
model = Net()
# Поместим нашу модель на GPU, если подключен.
model.to(device)

Net(
  (dp_three): Dropout(p=0.2, inplace=False)
  (dp_four): Dropout(p=0.2, inplace=False)
  (bn_one): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_one): Conv2d(3, 30, kernel_size=(3, 3), stride=(1, 1))
  (bn_two): BatchNorm2d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_two): Conv2d(30, 60, kernel_size=(3, 3), stride=(1, 1))
  (bn_three): BatchNorm2d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_three): Conv2d(60, 120, kernel_size=(3, 3), stride=(1, 1))
  (bn_four): BatchNorm2d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=480, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=60, bias=True)
  (out): Linear(in_features=60, out_features=100, bias=True)
)

In [7]:
# создадим optimizer и criterion
optimizer = optim.SGD(model.parameters(), lr=3e-3, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

In [8]:
def model_train_test(model, train_loader, test_loader, optimizer, criterion, epochs=10):
    # обучим и протестируем нашу самописную модель
    for epoch in tqdm(range(epochs)):  
        model.train()
        for data in train_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
        model.eval()
        loss_accumed = 0
        for data in test_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            output = model(inputs)
            loss = criterion(outputs, labels)
            loss_accumed += loss.item()
        
        print(f'\n epoch = {epoch + 1};    loss: {(loss_accumed / 10000):.3f}')

    print('\n Training is finished!')

In [9]:
# обучим и протестируем 
model_train_test(model, train_loader, test_loader, optimizer, criterion, epochs=10)

 10%|█         | 1/10 [00:13<01:57, 13.03s/it]


 epoch = 1;    loss: 0.314


 20%|██        | 2/10 [00:25<01:41, 12.69s/it]


 epoch = 2;    loss: 0.352


 30%|███       | 3/10 [00:38<01:28, 12.65s/it]

...


 epoch = 8;    loss: 0.405


 90%|█████████ | 9/10 [01:55<00:12, 12.85s/it]


 epoch = 9;    loss: 0.450


100%|██████████| 10/10 [02:07<00:00, 12.74s/it]


 epoch = 10;    loss: 0.421

 Training is finished!


### 2. Обучение классификатора картинок на примере CIFAR-100 (датасет можно изменить) через дообучение ImageNet Resnet-50

In [10]:
from torchvision import models

In [11]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [12]:
# Необходимые трансформации
mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]

normalize = transforms.Normalize(mean, std)

transforms = transforms.Compose([transforms.ToTensor(),
                                 normalize])

In [13]:
# Загрузим датасет CIFAR-100, сразу же создадим dataloader для него
# Если вам не хватает вычислительных ресурсов, то можно вернуться к CIFAR-10
train_dataset = torchvision.datasets.CIFAR100(root='data/',
                                             train=True,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True)

print(train_dataset)
print('='*85)
print(test_dataset)

train_dataset = MyOwnCifar(train_dataset, transforms)
test_dataset = MyOwnCifar(test_dataset, transforms)

Files already downloaded and verified
Files already downloaded and verified
Dataset CIFAR100
    Number of datapoints: 50000
    Root location: data/
    Split: Train
Dataset CIFAR100
    Number of datapoints: 10000
    Root location: ./data
    Split: Test


In [14]:
train_loader =  torch.utils.data.DataLoader(train_dataset,
                                            batch_size=64, 
                                            shuffle=True)

test_loader =  torch.utils.data.DataLoader(test_dataset, 
                                           batch_size=16,
                                           shuffle=False)

In [15]:
model = models.resnet50(pretrained=True)
set_parameter_requires_grad(model, True)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [16]:
model.to(device)

num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 1024)
model.fc = nn.Sequential(
    torch.nn.Dropout(0.5),
    torch.nn.Linear(num_ftrs, 1024),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(1024, 512),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(512, 256),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(256, 128),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(128, 100)
)

model.fc.to(device)

Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=2048, out_features=1024, bias=True)
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=1024, out_features=512, bias=True)
  (4): Dropout(p=0.2, inplace=False)
  (5): Linear(in_features=512, out_features=256, bias=True)
  (6): Dropout(p=0.2, inplace=False)
  (7): Linear(in_features=256, out_features=128, bias=True)
  (8): Dropout(p=0.2, inplace=False)
  (9): Linear(in_features=128, out_features=100, bias=True)
)

In [17]:
optimizer = optim.SGD(model.parameters(), lr=3e-3, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

In [18]:
# обучим и протестируем
model_train_test(model, train_loader, test_loader, optimizer, criterion, epochs=10)

 10%|█         | 1/10 [00:31<04:39, 31.01s/it]


 epoch = 1;    loss: 0.323


 20%|██        | 2/10 [01:01<04:07, 30.95s/it]


 epoch = 2;    loss: 0.328


 30%|███       | 3/10 [01:32<03:36, 30.94s/it]

...


 epoch = 8;    loss: 0.365


 90%|█████████ | 9/10 [04:44<00:32, 32.41s/it]


 epoch = 9;    loss: 0.369


100%|██████████| 10/10 [05:15<00:00, 31.51s/it]


 epoch = 10;    loss: 0.366

 Training is finished!


### 3. Обучение классификатора картинок на примере CIFAR-100 (датасет можно изменить) через дообучение ImageNet Resnet-50 с аугментацией (самописной, с использованием Pytorch встроенных методов)

In [21]:
# Загрузим датасет CIFAR-100, сразу же создадим dataloader для него
# Если вам не хватает вычислительных ресурсов, то можно вернуться к CIFAR-10
train_dataset = torchvision.datasets.CIFAR100(root='data/',
                                             train=True,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True)

# сделаем аугментацию dataset
mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]
trans_actions = transforms.Compose([transforms.Resize(44),
                                    transforms.RandomHorizontalFlip(p=0.5),
                                    transforms.RandomCrop(32, padding=4),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean, std)
                                    ])
test_transforms = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize(mean, std)
                                     ])

train_dataset = MyOwnCifar(train_dataset, trans_actions)
test_dataset = MyOwnCifar(test_dataset, test_transforms)

train_loader =  torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=64, 
                                            shuffle=True)

test_loader =  torch.utils.data.DataLoader(test_dataset, 
                                           batch_size=16,
                                           shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [22]:
# посмотрим на написанную сеть
model = Net()
# Поместим нашу модель на GPU, если подключен.
model.to(device)

# создадим optimizer и criterion
optimizer = optim.SGD(model.parameters(), lr=3e-3, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

# обучим и протестируем 
model_train_test(model, train_loader, test_loader, optimizer, criterion, epochs=10)

 10%|█         | 1/10 [00:28<04:12, 28.07s/it]


 epoch = 1;    loss: 0.306


 20%|██        | 2/10 [00:56<03:47, 28.44s/it]


 epoch = 2;    loss: 0.332


 30%|███       | 3/10 [01:25<03:20, 28.60s/it]

...


 epoch = 8;    loss: 0.354


 90%|█████████ | 9/10 [04:11<00:27, 27.80s/it]


 epoch = 9;    loss: 0.352


100%|██████████| 10/10 [04:39<00:00, 27.95s/it]


 epoch = 10;    loss: 0.372

 Training is finished!


In [23]:
# посмотрим на ImageNet Resnet-50 с аугментацией 
model = models.resnet50(pretrained=True)
set_parameter_requires_grad(model, True)

model.to(device)

num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 1024)
model.fc = nn.Sequential(
    torch.nn.Dropout(0.5),
    torch.nn.Linear(num_ftrs, 1024),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(1024, 512),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(512, 256),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(256, 128),
    torch.nn.Dropout(0.2),
    torch.nn.Linear(128, 100)
)

model.fc.to(device)

optimizer = optim.SGD(model.parameters(), lr=3e-3, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

# обучим и протестируем 
model_train_test(model, train_loader, test_loader, optimizer, criterion, epochs=10)

 10%|█         | 1/10 [00:40<06:05, 40.64s/it]


 epoch = 1;    loss: 0.315


 20%|██        | 2/10 [01:21<05:26, 40.81s/it]


 epoch = 2;    loss: 0.336


 30%|███       | 3/10 [02:01<04:44, 40.63s/it]

...


 epoch = 8;    loss: 0.349


 90%|█████████ | 9/10 [06:04<00:40, 40.29s/it]


 epoch = 9;    loss: 0.350


100%|██████████| 10/10 [06:43<00:00, 40.39s/it]


 epoch = 10;    loss: 0.368

 Training is finished!
